In [ ]:
import pandas as pd
import math

# Чтение датасетов из файлов
log = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Unit 3/log.xls', header=None)
us = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Unit 3/users.xls', encoding='koi8_r', sep='\t')

# Предобработка названий колонок
us.columns = ['user_id', 'email', 'city']
log.columns = ['user_id', 'time', 'bet', 'win']

# Функции
def func(row):
  """Функция учитывает ставки и выигрыши"""
  if row.win != 0:
    return row.win
  elif row.bet == 0 and row.win == 0:
    return row.bet
  elif row.bet != 0 and row.win == 0:
    return -row.bet


def net_func(row):
  """Функция используется для верного заполнения столбца выигрыша/проигрыша"""
  if row.win < 0:
    return row.win
  else:
    return row.win - row.bet


# Приведем признак user_id к одному формату в обоих датасетах  
us.user_id = us.user_id.apply(lambda x: x.lower())

# Избавимся от ошибок в user_id  
log = log[log.user_id != '#error']  
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])

# Объединяем два датасета в один большой
casino = pd.merge(log, us, on='user_id')

# Убираем лишний значок в столбце с датой и временем
casino.time = casino.time.str.replace('[', '')

# Применяем функцию к датасету в правильном заполнении значениями ставок и выигрыша
casino = casino.fillna(0)
new_win = casino.apply(lambda row: func(row), axis=1)
casino.win = new_win

# Добавляем столбец выигрыша/проигрыша, применив предварительно функцию
new_net = casino.apply(lambda row: net_func(row), axis=1)
casino['net'] = new_net

# Меняем последовательность столбцов
casino = casino[['user_id', 'time', 'bet', 'win', 'net', 'email', 'city']]

# Преобразуем датасет для работы со временем
casino.time = pd.to_datetime(casino.time)



####################### ВЫПОЛНЕНИЕ ЗАДАНИЙ #######################

# Считаем медиану баланса по каждому пользователю
math.trunc(casino.groupby('user_id').net.sum().median())

# Считаем наибольший суммарный выигрыш по городу после Москвы
casino.groupby('city').win.sum().sort_values(ascending=False).index[1]

# Считаем частное средних значений ставок по городам
math.trunc(casino[casino.bet != 0].groupby('city').bet.mean().max() / \
casino[casino.bet != 0].groupby('city').bet.mean().min())

# Считает, сколько раз пользователи приходили в каждом городе
casino.groupby('city').user_id.count()

## Считаем среднее количество приходов без ставок при условии, что была сделана хотя бы одна
# создаем два датафрейма для подсчета случаев нулевой ставки, затем объединяем их
group_null = casino[casino.bet == 0].groupby('user_id').bet.count()
group_not_null = casino[casino.bet > 0].groupby('user_id').bet.count()
joined = pd.merge(group_null, group_not_null, on='user_id')
joined
# Оставим только те строки, в которых у посетителя была хотя бы одна ставка, посчитаем среднее без ставки
joined = joined[joined['bet_y'] > 0]
math.trunc(joined['bet_x'].sum() / len(joined))

## Считаем, сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой
group_null = casino[casino.bet == 0].groupby('user_id').time.min()
group_not_null = casino[casino.bet > 0].groupby('user_id').time.min()
joined = pd.merge(group_null, group_not_null, on='user_id')
timedelta = (joined.time_y - joined.time_x).sum() / joined.shape[0]
timedelta.days